In [1]:
from symspellpy import SymSpell, Verbosity
import string
import re
import pandas as pd
import numpy as np

# Create a SymSpell object
sym_spell = SymSpell()

#Load dictionary
dictionary_path = 'nl_with_corpus_nos.txt'
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

True

In [2]:
# Define the sentence you want to correct
sentence = "Ben je in staat om zinnen te verbetren?"
# Set the maximum edit distance to consider for spelling corrections
max_edit_distance = 2

# Get suggestions for the misspelled words
suggestions = sym_spell.lookup_compound(sentence, max_edit_distance, ignore_non_words=True, ignore_term_with_digits=True, transfer_casing=True)
for suggestion in suggestions:
    print(f"Suggested Correction: {suggestion._term}")
print('Yes, But cant keep interpunction.. so we preprocess')


Suggested Correction: Ben je in staat om zinnen te verbeteren
Yes, But cant keep interpunction.. so we preprocess


In [3]:
#Preprocess the text so words interpunction remains after correction
def preprocess_text(text):
    # Separate words and punctuation (including spaces)
    words_and_punctuation = re.findall(r"[\w']+|[.,!:?;()\-]| +", text)

    # Separate punctuation from words
    processed_words = []
    for token in words_and_punctuation:

        if token.isalpha():
            corrected_word = sym_spell.lookup_compound(token, max_edit_distance, ignore_non_words=True, ignore_term_with_digits=True, transfer_casing=True)
            if corrected_word:
                processed_words.append(corrected_word[0].term)
            else:
                processed_words.append(token)
        else:
            processed_words.append(token)

    # Rejoin words and punctuation back into text
    processed_text = "".join(processed_words)
    return processed_text

In [4]:
#check if the special characters remain in place
preprocess_text('Ben je in staat: () om zinnen te verbetren?')

'Ben je in staat: () om zinnen te verbeteren?'

In [7]:
import pandas as pd
#load the corpus that needs to be spellchecked for suggestions. Made in vocabulary
preprocessed_sentence_df = pd.read_csv('E:/Data_exploration/GitHub/paraphraser_code/data_preparation/careplan_sentences_preprocessed.csv')

In [8]:
# Initialize SymSpell
sym_spell = SymSpell()
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# Define the preprocess_text function
def preprocess_text_save_edits(text):
    # Separate words and punctuation (including spaces)
    if not isinstance(text, str):
        return np.nan, np.nan, np.nan
    words_and_punctuation = re.findall(r"[\w']+|[.,!?;]| +", text)

    # Separate punctuation from words
    original_words = []
    processed_words = []
    final_sentence = []
    for token in words_and_punctuation:
        if token.isalpha():
            try:
                corrected_word = sym_spell.lookup_compound(token, max_edit_distance, ignore_non_words=True, ignore_term_with_digits=True, transfer_casing=True)
            #print(token)
                if corrected_word[0].term != token:
                    #print(corrected_word[0].term)
                    original_words.append(token)
                    processed_words.append(corrected_word[0].term)
                    #original_words.append(token)
                    final_sentence.append(corrected_word[0].term)
                else:
                    #original_words.append(token)
                    final_sentence.append(token)
            except:
                processed_words.append(np.nan)
                original_words.append(np.nan)
        else:
            #original_words.append(token)
            final_sentence.append(token)

    # Rejoin words and punctuation back into text
    final_text = "".join(final_sentence)
    return final_text, original_words, processed_words
#preprocessed_sentence_df = preprocessed_sentence_df.head(100)

# Example DataFrame with text column 'text_column_name'
preprocessed_sentence_df['processed_sentences'], preprocessed_sentence_df['original_words'], preprocessed_sentence_df['altered_words'] = zip(*preprocessed_sentence_df['sentences'].apply(preprocess_text_save_edits))

# Save the DataFrame with the preprocessed text and original words to a new CSV file
#preprocessed_sentence_df.to_csv('spellcheck_applied_sentences_corpus_nos.csv')


In [9]:
preprocessed_sentence_df.to_csv('spellcheck_applied_sentences_corpus_nos.csv')

OSError: [Errno 28] No space left on device

In [10]:
#preprocessed_sentence_df = pd.read_csv("spellcheck_applied_sentences_corpus_nos.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'spellcheck_applied_sentences_nos.csv'

In [10]:
no_nan_df = preprocessed_sentence_df[~preprocessed_sentence_df['altered_words'].isna()]
pd.set_option('display.max_colwidth', None)

lengths = no_nan_df['altered_words'].apply(len)

# Create a boolean mask for filtering based on non-empty lists
non_empty_mask = lengths != 0

# Filter the DataFrame based on the boolean mask
filtered_df = no_nan_df[non_empty_mask]

filtered_df.head()

,num_words,sentences,processed_sentences,original_words,altered_words
8,9,Mw De Hoog- pluym is gisteren opgenomen op afd.,Mw De Hoog pluim is gisteren opgenomen op afd.,[pluym],[pluim]
10,15,Zij is bekend met korte termijn geheugeverlies haar gehoord is zeer slecht heeft hiervoor hoorapparaten.,Zij is bekend met korte termijn geheugenverlies haar gehoord is zeer slecht heeft hiervoor hoorapparaten.,[geheugeverlies],[geheugenverlies]
14,3,Zij toileteerd zelfstandig.,Zij toile teerd zelfstandig.,[toileteerd],[toile teerd]
17,12,van der Vliet van Baarle is vandaag op afd 3 komen wonen.,van der Vliet van Baarde is vandaag op afd 3 komen wonen.,[Baarle],[Baarde]
23,16,Omdat zij lang niet heeft kunnen lopen met haar rollator is de sta/loop funtie van Mw.,Omdat zij lang niet heeft kunnen lopen met haar rollator is de staloop functie van Mw.,[funtie],[functie]


In [20]:
from collections import defaultdict
word_mapping = defaultdict(list)

# Iterate through each row in the DataFrame
for _, row in filtered_df.iterrows():
    altered_words = row['altered_words']
    original_words = row['original_words']

    # Iterate through the words and their corresponding suggestions in each row
    for altered_word, original_word in zip(altered_words, original_words):
        # Check if the altered_word already exists as a key in the dictionary
        if altered_word in word_mapping:
            # If it exists, append the original_word to the list of suggestions for that key
            if original_word not in word_mapping[altered_word]:
                word_mapping[altered_word].append(original_word)
        else:
            # If it doesn't exist, create a new entry in the dictionary with the altered_word as the key
            word_mapping[altered_word] = [original_word]

# Convert the defaultdict to a regular dictionary
word_mapping = dict(word_mapping)

In [21]:
word_mapping

{'pluim': ['pluym'],
 'geheugenverlies': ['geheugeverlies',
  'geheuegnverlies',
  'geheugfenverlies',
  'geheugensverlies'],
 'toile teerd': ['toileteerd'],
 'Baarde': ['Baarle'],
 'functie': ['funtie',
  'funktie',
  'fuctie',
  'funcite',
  'funcie',
  'flectie',
  'functio',
  'funcktie',
  'furcatie',
  'fucntie',
  'unctie',
  'fubctie',
  'dunctie',
  'functte',
  'finctie',
  'fnnctie',
  'gunctie',
  'funstie',
  'functiie',
  'fundtie',
  'functue',
  'funtctie',
  'fluctie',
  'funcrtie',
  'functig',
  'funmctie',
  'functiue'],
 'Mijl': ['Uijl'],
 'Arend': ['Arends'],
 'mentaalwelbevinden': ['mwntaalwelbevinden',
  'mentaalwelbinden',
  'mentaanwelbevinden',
  'metaalwelbevinden',
  'mentraalwelbevinden'],
 'ik': ['iki',
  'iak',
  'iok',
  'ilk',
  'ipk',
  'bik',
  'ík',
  'ikn',
  'aik',
  'uik',
  'iïk',
  'oik',
  'ikz',
  'gik',
  'ikl',
  'ikw',
  'Oik',
  'imk',
  'ikm',
  'jk',
  'ìk',
  'ihk',
  'Bik',
  'iko',
  'ikk',
  'zik',
  'gk',
  'ikd',
  'òòk',
  'ikg',

In [22]:
import pickle
with open('spelling_corrected_corpus_nos_suggestions.pickle', 'wb') as pickle_file:
    pickle.dump(word_mapping, pickle_file)